In [ ]:
import polars as pl
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pl.read_csv('/content/drive/MyDrive/retail_sales.csv')
df

product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
str,str,str,f64,f64,f64,f64,str,str,str,str,str,str
"""P0001""","""S0002""","""2017-01-02""",0.0,0.0,8.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0012""","""2017-01-02""",1.0,5.3,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0013""","""2017-01-02""",2.0,10.59,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0023""","""2017-01-02""",0.0,0.0,6.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0025""","""2017-01-02""",0.0,0.0,1.0,6.25,"""PR14""",null,"""PR03""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…
"""P0748""","""S0125""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0126""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0131""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null


In [ ]:
df['promo_bin_1'].value_counts()

promo_bin_1,count
str,u32
"""moderate""",418880
"""low""",551829
"""veryhigh""",255225
null,16803149
"""verylow""",1117148
"""high""",308607


In [ ]:
df['date'] = pl.to_datetime(df['date'])


In [ ]:
import seaborn as sns


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
import polars as pl

# Identify numeric columns (adjust types if needed)
numeric_cols = [col for col, dtype in df.schema.items() if dtype in (pl.Int64, pl.Float64)]

# Build conditions to filter out outliers using the IQR method for each numeric column
conditions = []
for col in numeric_cols:
    # Calculate Q1 and Q3 for the column
    q1 = df.select(pl.col(col).quantile(0.25)).item()
    q3 = df.select(pl.col(col).quantile(0.75)).item()
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    conditions.append((pl.col(col) >= lower_bound) & (pl.col(col) <= upper_bound))

# Combine conditions (a row is kept only if all numeric columns are within bounds)
combined_condition = conditions[0]
for cond in conditions[1:]:
    combined_condition &= cond

# Filter the DataFrame to remove outliers
df_no_outliers = df.filter(combined_condition)

print(df_no_outliers)
df

shape: (12_560_367, 13)
┌────────────┬──────────┬────────────┬───────┬───┬────────────┬────────────┬───────────┬───────────┐
│ product_id ┆ store_id ┆ date       ┆ sales ┆ … ┆ promo_type ┆ promo_bin_ ┆ promo_dis ┆ promo_dis │
│ ---        ┆ ---      ┆ ---        ┆ ---   ┆   ┆ _2         ┆ 2          ┆ count_2   ┆ count_typ │
│ str        ┆ str      ┆ str        ┆ f64   ┆   ┆ ---        ┆ ---        ┆ ---       ┆ e_2       │
│            ┆          ┆            ┆       ┆   ┆ str        ┆ str        ┆ str       ┆ ---       │
│            ┆          ┆            ┆       ┆   ┆            ┆            ┆           ┆ str       │
╞════════════╪══════════╪════════════╪═══════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ P0001      ┆ S0002    ┆ 2017-01-02 ┆ 0.0   ┆ … ┆ PR03       ┆ null       ┆ null      ┆ null      │
│ P0001      ┆ S0023    ┆ 2017-01-02 ┆ 0.0   ┆ … ┆ PR03       ┆ null       ┆ null      ┆ null      │
│ P0001      ┆ S0025    ┆ 2017-01-02 ┆ 0.0   ┆ … ┆ PR03       ┆ nul

product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
str,str,str,f64,f64,f64,f64,str,str,str,str,str,str
"""P0001""","""S0002""","""2017-01-02""",0.0,0.0,8.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0012""","""2017-01-02""",1.0,5.3,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0013""","""2017-01-02""",2.0,10.59,0.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0023""","""2017-01-02""",0.0,0.0,6.0,6.25,"""PR14""",null,"""PR03""",null,null,null
"""P0001""","""S0025""","""2017-01-02""",0.0,0.0,1.0,6.25,"""PR14""",null,"""PR03""",null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…
"""P0748""","""S0125""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0126""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null
"""P0748""","""S0131""","""2019-12-29""",null,null,null,18.9,"""PR14""",null,"""PR03""",null,null,null


In [ ]:
cols_to_drop = [
    'promo_type_1', 'promo_bin_1',
    'promo_type_2', 'promo_bin_2',
    'promo_discount_2', 'promo_discount_type_2'
]
df = df.drop(cols_to_drop)

In [ ]:
from prophet import Prophet
df_pandas = df_pandas.rename(columns={"timestamp": "ds", "value": "y"})

# Fit Prophet model
model = Prophet()
model.fit(df_pandas)

# Make future predictions
future = model.make_future_dataframe(periods=30, freq="D")
forecast = model.predict(future)

print(forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]])